In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('original_dataset.csv')
df = df.rename(columns={'Kod': 'Course Code', 'Tanım': 'Course Title', 'Öğrenci No': 'Student Number', 'Bölüm': 'Department Code', 'Bölüm (tanım)': 'Course Level', 'Not': 'Letter Grade', 'Kayıt drm.': 'Status', 'Dönem': 'Semester'})
df.drop('Skala', axis=1, inplace=True)
df

,Course Code,Course Title,Student Number,Department Code,Course Level,Letter Grade,Status,Semester
0,IE 316,Engineering Economics,713,IE,Endüstri Mühendisliği Lisans Programı,A,Başarıyla tamamlandı,2014 - Bahar
1,MGT 201,Principles of Management,713,ITM,Uluslararası Ticaret ve İşletmecilik,A,Başarıyla tamamlandı,2013 - Güz
2,PHYS 104,Physics II-Electromagnetics & Modern phy,713,IE,Endüstri Mühendisliği Lisans Programı,A,Başarıyla tamamlandı,2012 - Bahar
3,UNI 123,Textual Analysis and Effective Com.,713,IE,Endüstri Mühendisliği Lisans Programı,A,Başarıyla tamamlandı,2012 - Bahar
4,ENGR 244,Engineering Materials,713,IE,Endüstri Mühendisliği Lisans Programı,A-,Başarıyla tamamlandı,2013 - Bahar
...,...,...,...,...,...,...,...,...
55470,ITM 406,Global Business Strategy,2315,MGT,İşletme Lisans Programı,C,Başarıyla tamamlandı,2014 - Bahar
55471,UNI 116,Turkish for International Students II,2315,MGT,İşletme Lisans Programı,C+,Başarıyla tamamlandı,2014 - Bahar
55472,ENGR 105,Introduction to Engineering,2315,MGT,İşletme Lisans Programı,IA,Başarısız tamamlandı,2014 - Bahar
55473,CS 100,Int. to Computer Science and Eng.,2315,MGT,İşletme Lisans Programı,S,Başarıyla tamamlandı,2014 - Bahar


In [3]:
df.insert(2, 'Course Year', '1')
for row_idx in df.index:
    course_code = df.iloc[row_idx, 0]
    if course_code.split()[1][0] in ['1', '2', '3', '4']:
        df.iloc[row_idx, 5] = 'Undergraduate'
    elif course_code.split()[1][0] in ['5', '6']:
        df.iloc[row_idx, 5] = 'Graduate'
    else:
        df.iloc[row_idx, 5] = 'PhD'
df

,Course Code,Course Title,Course Year,Student Number,Department Code,Course Level,Letter Grade,Status,Semester
0,IE 316,Engineering Economics,1,713,IE,Undergraduate,A,Başarıyla tamamlandı,2014 - Bahar
1,MGT 201,Principles of Management,1,713,ITM,Undergraduate,A,Başarıyla tamamlandı,2013 - Güz
2,PHYS 104,Physics II-Electromagnetics & Modern phy,1,713,IE,Undergraduate,A,Başarıyla tamamlandı,2012 - Bahar
3,UNI 123,Textual Analysis and Effective Com.,1,713,IE,Undergraduate,A,Başarıyla tamamlandı,2012 - Bahar
4,ENGR 244,Engineering Materials,1,713,IE,Undergraduate,A-,Başarıyla tamamlandı,2013 - Bahar
...,...,...,...,...,...,...,...,...,...
55470,ITM 406,Global Business Strategy,1,2315,MGT,Undergraduate,C,Başarıyla tamamlandı,2014 - Bahar
55471,UNI 116,Turkish for International Students II,1,2315,MGT,Undergraduate,C+,Başarıyla tamamlandı,2014 - Bahar
55472,ENGR 105,Introduction to Engineering,1,2315,MGT,Undergraduate,IA,Başarısız tamamlandı,2014 - Bahar
55473,CS 100,Int. to Computer Science and Eng.,1,2315,MGT,Undergraduate,S,Başarıyla tamamlandı,2014 - Bahar


In [4]:
for row_idx in df.index:
    semester_name = df.iloc[row_idx, 8].split(' - ')
    if semester_name[1] == 'Güz':
        df.iloc[row_idx, 8] = semester_name[0] + ' - ' + 'Fall'
    elif semester_name[1] == 'Bahar':
        df.iloc[row_idx, 8] = semester_name[0] + ' - ' + 'Spring'
    else:
        df.iloc[row_idx, 8] = semester_name[0] + ' - ' + 'Summer'
df

,Course Code,Course Title,Course Year,Student Number,Department Code,Course Level,Letter Grade,Status,Semester
0,IE 316,Engineering Economics,1,713,IE,Undergraduate,A,Başarıyla tamamlandı,2014 - Spring
1,MGT 201,Principles of Management,1,713,ITM,Undergraduate,A,Başarıyla tamamlandı,2013 - Fall
2,PHYS 104,Physics II-Electromagnetics & Modern phy,1,713,IE,Undergraduate,A,Başarıyla tamamlandı,2012 - Spring
3,UNI 123,Textual Analysis and Effective Com.,1,713,IE,Undergraduate,A,Başarıyla tamamlandı,2012 - Spring
4,ENGR 244,Engineering Materials,1,713,IE,Undergraduate,A-,Başarıyla tamamlandı,2013 - Spring
...,...,...,...,...,...,...,...,...,...
55470,ITM 406,Global Business Strategy,1,2315,MGT,Undergraduate,C,Başarıyla tamamlandı,2014 - Spring
55471,UNI 116,Turkish for International Students II,1,2315,MGT,Undergraduate,C+,Başarıyla tamamlandı,2014 - Spring
55472,ENGR 105,Introduction to Engineering,1,2315,MGT,Undergraduate,IA,Başarısız tamamlandı,2014 - Spring
55473,CS 100,Int. to Computer Science and Eng.,1,2315,MGT,Undergraduate,S,Başarıyla tamamlandı,2014 - Spring


In [5]:
for row_idx in df.index:        
    if df.iloc[row_idx, 7] == 'Başarıyla tamamlandı':
        df.iloc[row_idx, 7] = 'Successful'
    elif df.iloc[row_idx, 7] == 'Başarısız tamamlandı':
        df.iloc[row_idx, 7] = 'Unsuccesful'
    else:
        df.iloc[row_idx, 7] = 'Recorded'
df

,Course Code,Course Title,Course Year,Student Number,Department Code,Course Level,Letter Grade,Status,Semester
0,IE 316,Engineering Economics,1,713,IE,Undergraduate,A,Successful,2014 - Spring
1,MGT 201,Principles of Management,1,713,ITM,Undergraduate,A,Successful,2013 - Fall
2,PHYS 104,Physics II-Electromagnetics & Modern phy,1,713,IE,Undergraduate,A,Successful,2012 - Spring
3,UNI 123,Textual Analysis and Effective Com.,1,713,IE,Undergraduate,A,Successful,2012 - Spring
4,ENGR 244,Engineering Materials,1,713,IE,Undergraduate,A-,Successful,2013 - Spring
...,...,...,...,...,...,...,...,...,...
55470,ITM 406,Global Business Strategy,1,2315,MGT,Undergraduate,C,Successful,2014 - Spring
55471,UNI 116,Turkish for International Students II,1,2315,MGT,Undergraduate,C+,Successful,2014 - Spring
55472,ENGR 105,Introduction to Engineering,1,2315,MGT,Undergraduate,IA,Unsuccesful,2014 - Spring
55473,CS 100,Int. to Computer Science and Eng.,1,2315,MGT,Undergraduate,S,Successful,2014 - Spring


In [17]:
course_df = pd.read_csv('course_details.csv')
course_df

,Department Name,Course Code,Course Title,Prerequisites,Theoritical,Practical,Course Credits,ECTS
0,Political Science and International Relations ...,UNI 100,Exploring İstanbul,NaN,2,2,3,5
1,Political Science and International Relations ...,UNI 111,Critical Reading and Writing in Turkish I,NaN,3,0,3,5
2,Political Science and International Relations ...,UNI 117,Understanding Society and Culture,NaN,3,0,3,5
3,Political Science and International Relations ...,UNI 123,Textual Analysis and Effective Communication,NaN,3,0,3,5
4,Political Science and International Relations ...,UNI 221,World Civilizations and Global Encounters: Unt...,NaN,3,0,3,5
...,...,...,...,...,...,...,...,...
22777,Law,YBS 427,Coaching and Guidance in Business,NaN,3,0,3,5
22778,Law,YBS 428,Katılım Sigortacılığı,NaN,3,0,3,5
22779,Law,YBS 438,Global Management and Leadership Thought,NaN,3,0,3,5
22780,Law,YBS 446,CO-OP: Cooperative Learning I,NaN,3,0,3,5


In [18]:
course_df.drop_duplicates(subset=['Course Code'], inplace=True)
course_df.index = range(len(course_df))
course_df

,Department Name,Course Code,Course Title,Prerequisites,Theoritical,Practical,Course Credits,ECTS
0,Political Science and International Relations ...,UNI 100,Exploring İstanbul,NaN,2,2,3,5
1,Political Science and International Relations ...,UNI 111,Critical Reading and Writing in Turkish I,NaN,3,0,3,5
2,Political Science and International Relations ...,UNI 117,Understanding Society and Culture,NaN,3,0,3,5
3,Political Science and International Relations ...,UNI 123,Textual Analysis and Effective Communication,NaN,3,0,3,5
4,Political Science and International Relations ...,UNI 221,World Civilizations and Global Encounters: Unt...,NaN,3,0,3,5
...,...,...,...,...,...,...,...,...
1423,Law,HUK 315,Environmental Law,NaN,2,0,2,3
1424,Law,HUK 317,Criminology,NaN,2,0,2,3
1425,Law,HUK 324,Law of Security,NaN,2,0,2,3
1426,Law,MDB 497,Bitirme Projesi I,Senior standing and GPA at least 2.00,1,2,2,10


In [19]:
df.insert(9, 'Theoritical', 3)
df.insert(10, 'Practical', 0)
df.insert(11, 'Course Credit', 3)
df.insert(12, 'ECTS', 5)

ValueError: cannot insert Theoritical, already exists

In [20]:
unavailable_courses = set()
for row_idx in df.index:
    course_code = df.iloc[row_idx, 0]
    course_title = df.iloc[row_idx, 1]
    title_indices = list(course_df.iloc[:, 2][course_df.iloc[:, 2] == course_title].index)
    code_indices = list(course_df.iloc[:, 1][course_df.iloc[:, 1] == course_code].index) 
    if len(title_indices) > 1:
        title_index = title_indices[0]
        if course_title == course_df.iloc[title_index, 2]:
            df.iloc[row_idx, 0] = course_df.iloc[title_index, 1]
            df.iloc[row_idx, 9] = course_df.iloc[title_index, 4]
            df.iloc[row_idx, 10] = course_df.iloc[title_index, 5]
            df.iloc[row_idx, 11] = course_df.iloc[title_index, 6]
            df.iloc[row_idx, 12] = course_df.iloc[title_index, 7]
            continue
    elif len(code_indices) > 1:
        code_index = code_indices[0]
        if course_code == course_df.iloc[code_index, 1]:
            df.iloc[row_idx, 1] = course_df.iloc[code_index, 2]
            df.iloc[row_idx, 9] = course_df.iloc[code_index, 4]
            df.iloc[row_idx, 10] = course_df.iloc[code_index, 5]
            df.iloc[row_idx, 11] = course_df.iloc[code_index, 6]
            df.iloc[row_idx, 12] = course_df.iloc[code_index, 7]
    else: unavailable_courses.add((course_code, course_title))

In [21]:
for idx in df.index:
    df.iloc[idx, 2] = df.iloc[idx, 0].split()[1][0]
df

,Course Code,Course Title,Course Year,Student Number,Department Code,Course Level,Letter Grade,Status,Semester,Theoritical,Practical,Course Credit,ECTS
0,IE 316,Engineering Economics,3,713,IE,Undergraduate,A,Successful,2014 - Spring,3,0,3,5
1,MGT 201,Principles of Management,2,713,ITM,Undergraduate,A,Successful,2013 - Fall,3,0,3,5
2,PHYS 104,Physics II-Electromagnetics & Modern phy,1,713,IE,Undergraduate,A,Successful,2012 - Spring,3,0,3,5
3,UNI 123,Textual Analysis and Effective Com.,1,713,IE,Undergraduate,A,Successful,2012 - Spring,3,0,3,5
4,ENGR 244,Engineering Materials,2,713,IE,Undergraduate,A-,Successful,2013 - Spring,3,0,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55470,ITM 406,Global Business Strategy,4,2315,MGT,Undergraduate,C,Successful,2014 - Spring,3,0,3,5
55471,UNI 116,Turkish for International Students II,1,2315,MGT,Undergraduate,C+,Successful,2014 - Spring,3,0,3,5
55472,ENGR 105,Introduction to Engineering,1,2315,MGT,Undergraduate,IA,Unsuccesful,2014 - Spring,0,2,1,2
55473,CS 100,Int. to Computer Science and Eng.,1,2315,MGT,Undergraduate,S,Successful,2014 - Spring,3,0,3,5


In [22]:
print('The following courses are not offered now. Graduate Courses are not scraped from website', len(unavailable_courses))
for c, t in unavailable_courses:
    print(c, t)

The following courses are not offered now. Graduate Courses are not scraped from website 720
SOC 460 Society,Power and Justice in the M.E
HUK 552 Rekabet Hukukunun Temel Konuları II
SOC 511 Sociology of Everyday Life
HUK 511 Osmanlı Anayasa Hukuku
CTV 403 Documentary Making
ECE 571 Software Engineering
ECE 592A Independent Study I
BGM 545 Biyometrik Sis. ve Kimlik Dogrulama
HUK 410 Deniz Ticaret Hukuku
FRE 102 French II
SOC 425 Gender and Society
ARAB 103 Arabic III
LIT 433 Klasik Türk Edebiyatında Mesnevi
HUK 557 Yeni TTK'da Anonim Şirketlerde Yönetim
HUK 104 Medeni Hukuk II
HUK 560 Eser Üzerindeki Hakların İhlalinden
HIST 685 Read.and Resea.in the Hist.of Ott. M.
POLS 516 Islam and Pol.in the Midd.East
HUK 206 Ceza Hukuku Genel Hükümler II
ISE 548 Product Design and Innovation
EECS 202 EECS II: Systems Fundamentals
HUK 531 Elektrik Piyasalarının Regülasyonu
ECE 503 mm Wave IC Design
UNI 115 Turkish for International Students I
MGT 536 Bitirme Projesi
PSY 491 Selected Topics in Psycho

In [23]:
unweight_grades = []
for row_idx in df.index:
    if df.iloc[row_idx, 6] in ['IA', 'S', 'U', 'NP', 'TP']:
        unweight_grades.append(row_idx)        
for idx in unweight_grades:
    df.drop(idx, inplace=True)
df.index = range(len(df))
df

,Course Code,Course Title,Course Year,Student Number,Department Code,Course Level,Letter Grade,Status,Semester,Theoritical,Practical,Course Credit,ECTS
0,IE 316,Engineering Economics,3,713,IE,Undergraduate,A,Successful,2014 - Spring,3,0,3,5
1,MGT 201,Principles of Management,2,713,ITM,Undergraduate,A,Successful,2013 - Fall,3,0,3,5
2,PHYS 104,Physics II-Electromagnetics & Modern phy,1,713,IE,Undergraduate,A,Successful,2012 - Spring,3,0,3,5
3,UNI 123,Textual Analysis and Effective Com.,1,713,IE,Undergraduate,A,Successful,2012 - Spring,3,0,3,5
4,ENGR 244,Engineering Materials,2,713,IE,Undergraduate,A-,Successful,2013 - Spring,3,0,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49342,ITM 416,Special Topics in Inter.Trade,4,2315,MGT,Undergraduate,B-,Successful,2014 - Spring,3,0,3,5
49343,MGT 202,Organizational Behavior,2,2315,MGT,Undergraduate,B-,Successful,2014 - Spring,3,0,3,5
49344,ITM 310,Supply Chain Management,3,2315,MGT,Undergraduate,C,Successful,2014 - Spring,3,0,3,5
49345,ITM 406,Global Business Strategy,4,2315,MGT,Undergraduate,C,Successful,2014 - Spring,3,0,3,5


In [24]:
df.to_csv('cleaned_dataset.csv', index=False)